<a href="https://colab.research.google.com/github/yijiejiang/Google-Colab/blob/master/LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch

In [17]:
import random

创建默认标签

In [18]:
true_W1 = torch.tensor([2.323, -3.1134])
true_W2 = torch.tensor([10.231, -99.4])
true_b = 6.66

生成数据集

In [19]:
def synthetic_data(W1, W2, b, num_examples):
  X = torch.normal(0, 1, (num_examples, len(W1)))
  y = torch.matmul(X ** 2, W1) + torch.matmul(X, W2) + b
  y += torch.normal(0, 0.01, y.shape)
  return X, y.reshape((-1, 1))

生成数据与标签

In [20]:
features, labels = synthetic_data(true_W1, true_W2, true_b, 100000)

数据处理

In [21]:
def data_iter(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples))
  random.shuffle(indices)
  for i in range(0, num_examples, batch_size):
      batch_indices = torch.tensor(
          indices[i: min(i + batch_size, num_examples)])
      yield features[batch_indices], labels[batch_indices]

构建模型

In [22]:
def get_model(X, W1, W2, b):
  return torch.matmul(X ** 2, W1) + torch.matmul(X, W2) + b

In [23]:
net = get_model

初始化参数

In [24]:
def init_parameters():
  W1 = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
  W2 = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
  B = torch.zeros(1, requires_grad=True)
  return W1, W2, B

In [25]:
W1, W2, B = init_parameters()

创建损失函数

In [26]:
def get_loss(y_hat, y):
  """均方损失"""
  return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [27]:
loss = get_loss

创建优化器

In [28]:
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

开始训练

In [29]:
batch_size = 10
lr = 0.0001
for epoch in range(10):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, W1, W2, B), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([W1, W2, B], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, W1, W2, B), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

print(f'w的估计误差: {true_W1 - W1.reshape(true_W1.shape)}')
print(f'b的估计误差: {true_b - B}')

epoch 1, loss 675.207520
epoch 2, loss 91.239822
epoch 3, loss 12.655132
epoch 4, loss 1.889470
epoch 5, loss 0.336106
epoch 6, loss 0.079855
epoch 7, loss 0.025263
epoch 8, loss 0.009496
epoch 9, loss 0.003871
epoch 10, loss 0.001636
w的估计误差: tensor([-0.0220, -0.0221], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0784], grad_fn=<RsubBackward1>)


In [30]:
print(f'w: {W1.reshape(true_W1.shape)}')
print(f'b: {B}')

w: tensor([ 2.3450, -3.0913], grad_fn=<ReshapeAliasBackward0>)
b: tensor([6.5816], requires_grad=True)
